In [12]:
import torch
import json
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [5]:
stage1 = torch.load('/mnt/public02/usr/yuanpeiwen/instruction_pool_cleaned/pool_evolve/ap_outputs/no_dolly_normal_order_multiply_alpha05_lambda_09_gamma1_ap_Alpaca.pth')
stage2 = torch.load('/mnt/public02/usr/yuanpeiwen/instruction_pool_cleaned/pool_evolve/ap_outputs/no_dolly_normal_order_multiply_alpha05_lambda_09_gamma1_ap_ShareGPT.pth')
stage3 = torch.load('/mnt/public02/usr/yuanpeiwen/instruction_pool_cleaned/pool_evolve/ap_outputs/no_dolly_normal_order_multiply_alpha05_lambda_09_gamma1_ap_WizardLM_alpaca.pth')
stage4 = torch.load('/mnt/public02/usr/yuanpeiwen/instruction_pool_cleaned/pool_evolve/ap_outputs/no_dolly_normal_order_multiply_alpha05_lambda_09_gamma1_ap_WizardLM_sharegpt.pth')

# stage1 = torch.load('/mnt/public02/usr/yuanpeiwen/instruction_pool_cleaned/baselines/deita_outputs/no_dolly_normal_order_sbs_deita_Alpaca.pth')
# stage2 = torch.load('/mnt/public02/usr/yuanpeiwen/instruction_pool_cleaned/baselines/deita_outputs/no_dolly_normal_order_sbs_deita_ShareGPT.pth')
# stage3 = torch.load('/mnt/public02/usr/yuanpeiwen/instruction_pool_cleaned/baselines/deita_outputs/no_dolly_normal_order_sbs_deita_WizardLM_alpaca.pth')
# stage4 = torch.load('/mnt/public02/usr/yuanpeiwen/instruction_pool_cleaned/baselines/deita_outputs/no_dolly_normal_order_sbs_deita_WizardLM_sharegpt.pth')

In [6]:
len_stage1 = []
for sample in stage1['data']:
    conversatios = sample['conversations']
    for turn in conversatios:
        if turn['from'] in ['system', 'human', 'user']:
            continue
        len_stage1.append(len(turn['value'].strip().split(' ')))

print(sum(len_stage1) / len(len_stage1))

len_stage2 = []
for sample in stage2['data']:
    conversatios = sample['conversations']
    for turn in conversatios:
        if turn['from'] in ['system', 'human', 'user']:
            continue
        len_stage2.append(len(turn['value'].strip().split(' ')))

print(sum(len_stage2) / len(len_stage2))

len_stage3 = []
for sample in stage3['data']:
    conversatios = sample['conversations']
    for turn in conversatios:
        if turn['from'] in ['system', 'human', 'user']:
            continue
        len_stage3.append(len(turn['value'].strip().split(' ')))

print(sum(len_stage3) / len(len_stage3))

len_stage4 = []
for sample in stage4['data']:
    conversatios = sample['conversations']
    for turn in conversatios:
        if turn['from'] in ['system', 'human', 'user']:
            continue
        len_stage4.append(len(turn['value'].strip().split(' ')))
print(sum(len_stage4) / len(len_stage4))

188.44916666666666
244.36088419029312
271.0185873605948
270.4294405344149


In [7]:
filtered_out_data_12 = []
filtered_out_data_23 = []
filtered_out_data_34 = []

add_in_data_12 = []
add_in_data_23 = []
add_in_data_34 = []

In [8]:
embedding_hash = []

In [9]:
stage1_no_tensor = []
for sample in stage1['data']:
    sample['idx'] = len(embedding_hash)
    embedding_hash.append(sample['embedding'])
    del sample['embedding']
    stage1_no_tensor.append(sample)

stage2_no_tensor = []
for sample in stage2['data']:
    sample['idx'] = len(embedding_hash)
    embedding_hash.append(sample['embedding'])
    del sample['embedding']
    stage2_no_tensor.append(sample)

stage3_no_tensor = []
for sample in stage3['data']:
    sample['idx'] = len(embedding_hash)
    embedding_hash.append(sample['embedding'])
    del sample['embedding']
    stage3_no_tensor.append(sample)

stage4_no_tensor = []
for sample in stage4['data']:
    sample['idx'] = len(embedding_hash)
    embedding_hash.append(sample['embedding'])
    del sample['embedding']
    stage4_no_tensor.append(sample)


for sample in stage1_no_tensor:
    if sample not in stage2_no_tensor:
        filtered_out_data_12.append(sample)
for sample in stage2_no_tensor:
    if sample not in stage1_no_tensor:
        add_in_data_12.append(sample)

for sample in stage2_no_tensor:
    if sample not in stage3_no_tensor:
        filtered_out_data_23.append(sample)
for sample in stage3_no_tensor:
    if sample not in stage2_no_tensor:
        add_in_data_23.append(sample)

for sample in stage3_no_tensor:
    if sample not in stage4_no_tensor:
        filtered_out_data_34.append(sample)
for sample in stage4_no_tensor:
    if sample not in stage3_no_tensor:
        add_in_data_34.append(sample)

In [10]:
for i in range(len(filtered_out_data_12)):
    filtered_out_data_12[i]['embedding'] = embedding_hash[filtered_out_data_12[i]['idx']]
    add_in_data_12[i]['embedding'] = embedding_hash[add_in_data_12[i]['idx']]
for i in range(len(filtered_out_data_23)):   
    filtered_out_data_23[i]['embedding'] = embedding_hash[filtered_out_data_23[i]['idx']]
    add_in_data_23[i]['embedding'] = embedding_hash[add_in_data_23[i]['idx']]
for i in range(len(filtered_out_data_34)):
    filtered_out_data_34[i]['embedding'] = embedding_hash[filtered_out_data_34[i]['idx']]
    add_in_data_34[i]['embedding'] = embedding_hash[add_in_data_34[i]['idx']]


In [27]:
complexity = []
quality = []
instruction_len = []
response_len = []
for sample in filtered_out_data_12:
    flag1 = False
    flag2 = False
    tmp_instruction_len = []
    tmp_response_len = []
    conversations = sample['conversations']
    for turn in conversations:
        if turn['from'] == 'system':
            continue
        if turn['from'] in ['human', 'user']:
            tmp_instruction_len.append(len(turn['value'].strip().split(' ')))
            flag2 = True
        else:
            tmp_response_len.append(len(turn['value'].strip().split(' ')))
            flag1 = True
    if flag1 and flag2:
        complexity.append(sample['complexity'])
        quality.append(sample['quality'])
        instruction_len.append(sum(tmp_instruction_len)/len(tmp_instruction_len))
        response_len.append(sum(tmp_response_len)/len(tmp_response_len))
print(f'Complexity: {sum(complexity)/len(complexity)}')
print(f'Quality: {sum(quality)/len(quality)}')
print(f'Instruction Len: {sum(instruction_len)/len(instruction_len)}')
print(f'Response Len: {sum(response_len)/len(response_len)}')

complexity = []
quality = []
instruction_len = []
response_len = []
for sample in add_in_data_12:
    flag1 = False
    flag2 = False
    tmp_instruction_len = []
    tmp_response_len = []
    conversations = sample['conversations']
    for turn in conversations:
        if turn['from'] == 'system':
            continue
        if turn['from'] in ['human', 'user']:
            tmp_instruction_len.append(len(turn['value'].strip().split(' ')))
            flag2 = True
        else:
            tmp_response_len.append(len(turn['value'].strip().split(' ')))
            flag1 = True
    if flag1 and flag2:
        complexity.append(sample['complexity'])
        quality.append(sample['quality'])
        instruction_len.append(sum(tmp_instruction_len)/len(tmp_instruction_len))
        response_len.append(sum(tmp_response_len)/len(tmp_response_len))
print(f'Complexity: {sum(complexity)/len(complexity)}')
print(f'Quality: {sum(quality)/len(quality)}')
print(f'Instruction Len: {sum(instruction_len)/len(instruction_len)}')
print(f'Response Len: {sum(response_len)/len(response_len)}')

Complexity: 1.951808860616201
Quality: 4.403591043384498
Instruction Len: 16.812833333333334
Response Len: 188.44916666666666
Complexity: 2.825256210936916
Quality: 4.357931977857371
Instruction Len: 291.2171636842078
Response Len: 260.29975346450846


In [28]:
complexity = []
quality = []
instruction_len = []
response_len = []
for sample in filtered_out_data_23:
    flag1 = False
    flag2 = False
    tmp_instruction_len = []
    tmp_response_len = []
    conversations = sample['conversations']
    for turn in conversations:
        if turn['from'] == 'system':
            continue
        if turn['from'] in ['human', 'user']:
            tmp_instruction_len.append(len(turn['value'].strip().split(' ')))
            flag2 = True
        else:
            tmp_response_len.append(len(turn['value'].strip().split(' ')))
            flag1 = True
    if flag1 and flag2:
        complexity.append(sample['complexity'])
        quality.append(sample['quality'])
        instruction_len.append(sum(tmp_instruction_len)/len(tmp_instruction_len))
        response_len.append(sum(tmp_response_len)/len(tmp_response_len))
print(f'Complexity: {sum(complexity)/len(complexity)}')
print(f'Quality: {sum(quality)/len(quality)}')
print(f'Instruction Len: {sum(instruction_len)/len(instruction_len)}')
print(f'Response Len: {sum(response_len)/len(response_len)}')

complexity = []
quality = []
instruction_len = []
response_len = []
for sample in add_in_data_23:
    flag1 = False
    flag2 = False
    tmp_instruction_len = []
    tmp_response_len = []
    conversations = sample['conversations']
    for turn in conversations:
        if turn['from'] == 'system':
            continue
        if turn['from'] in ['human', 'user']:
            tmp_instruction_len.append(len(turn['value'].strip().split(' ')))
            flag2 = True
        else:
            tmp_response_len.append(len(turn['value'].strip().split(' ')))
            flag1 = True
    if flag1 and flag2:
        complexity.append(sample['complexity'])
        quality.append(sample['quality'])
        instruction_len.append(sum(tmp_instruction_len)/len(tmp_instruction_len))
        response_len.append(sum(tmp_response_len)/len(tmp_response_len))
print(f'Complexity: {sum(complexity)/len(complexity)}')
print(f'Quality: {sum(quality)/len(quality)}')
print(f'Instruction Len: {sum(instruction_len)/len(instruction_len)}')
print(f'Response Len: {sum(response_len)/len(response_len)}')

Complexity: 2.825256210936916
Quality: 4.357931977857371
Instruction Len: 291.2171636842078
Response Len: 260.29975346450846
Complexity: 4.015326088705995
Quality: 4.606013654295061
Instruction Len: 275.6666785972795
Response Len: 299.96230599484863


In [30]:
complexity = []
quality = []
instruction_len = []
response_len = []
for sample in filtered_out_data_34:
    flag1 = False
    flag2 = False
    tmp_instruction_len = []
    tmp_response_len = []
    conversations = sample['conversations']
    for turn in conversations:
        if turn['from'] == 'system':
            continue
        if turn['from'] in ['human', 'user']:
            tmp_instruction_len.append(len(turn['value'].strip().split(' ')))
            flag2 = True
        else:
            tmp_response_len.append(len(turn['value'].strip().split(' ')))
            flag1 = True
    if flag1 and flag2:
        complexity.append(sample['complexity'])
        quality.append(sample['quality'])
        instruction_len.append(sum(tmp_instruction_len)/len(tmp_instruction_len))
        response_len.append(sum(tmp_response_len)/len(tmp_response_len))
print(f'Complexity: {sum(complexity)/len(complexity)}')
print(f'Quality: {sum(quality)/len(quality)}')
print(f'Instruction Len: {sum(instruction_len)/len(instruction_len)}')
print(f'Response Len: {sum(response_len)/len(response_len)}')

complexity = []
quality = []
instruction_len = []
response_len = []
for sample in add_in_data_34:
    flag1 = False
    flag2 = False
    tmp_instruction_len = []
    tmp_response_len = []
    conversations = sample['conversations']
    for turn in conversations:
        if turn['from'] == 'system':
            continue
        if turn['from'] in ['human', 'user']:
            tmp_instruction_len.append(len(turn['value'].strip().split(' ')))
            flag2 = True
        else:
            tmp_response_len.append(len(turn['value'].strip().split(' ')))
            flag1 = True
    if flag1 and flag2:
        complexity.append(sample['complexity'])
        quality.append(sample['quality'])
        instruction_len.append(sum(tmp_instruction_len)/len(tmp_instruction_len))
        response_len.append(sum(tmp_response_len)/len(tmp_response_len))
print(f'Complexity: {sum(complexity)/len(complexity)}')
print(f'Quality: {sum(quality)/len(quality)}')
print(f'Instruction Len: {sum(instruction_len)/len(instruction_len)}')
print(f'Response Len: {sum(response_len)/len(response_len)}')

Complexity: 4.015326088705995
Quality: 4.606013654295061
Instruction Len: 275.6666785972795
Response Len: 299.96230599484863
Complexity: 4.295817918055113
Quality: 4.70867566511273
Instruction Len: 307.42572071904686
Response Len: 295.5129821769998


In [29]:
embeddings = torch.cat([item['embedding'].unsqueeze(0) for item in filtered_out_data_12], dim=0).cuda()

similarity_matrix = torch.zeros((embeddings.size(0), embeddings.size(0)))
batch_size = 1  # 可以根据显存大小调整

# 逐批计算
for i in range(0, embeddings.size(0), batch_size):
    end_i = i + batch_size
    batch_i = embeddings[i:end_i]
    similarity = F.cosine_similarity(batch_i.unsqueeze(1), embeddings.unsqueeze(0), dim=-1)
    similarity_matrix[i:end_i, :] = similarity

mask = torch.ones(similarity_matrix.shape, dtype=bool)
mask.fill_diagonal_(0)
filtered_similarity = similarity_matrix.masked_select(mask).view(embeddings.size(0), -1)
nn_sim = filtered_similarity.max(dim=1).values
quality = [item['quality'] for item in filtered_out_data_12]
complexity = [item['complexity'] for item in filtered_out_data_12]
print(f"Filtered out 12: quality: {sum(quality) / len(quality)} -- complexity: {sum(complexity) / len(complexity)} -- avg similarity: {filtered_similarity.mean()} -- avg nearest neighbour similarity: {nn_sim.mean()}")

embeddings = torch.cat([item['embedding'].unsqueeze(0) for item in add_in_data_12], dim=0).cuda()

similarity_matrix = torch.zeros((embeddings.size(0), embeddings.size(0)))
batch_size = 1  # 可以根据显存大小调整

# 逐批计算
for i in range(0, embeddings.size(0), batch_size):
    end_i = i + batch_size
    batch_i = embeddings[i:end_i]
    similarity = F.cosine_similarity(batch_i.unsqueeze(1), embeddings.unsqueeze(0), dim=-1)
    similarity_matrix[i:end_i, :] = similarity

mask = torch.ones(similarity_matrix.shape, dtype=bool)
mask.fill_diagonal_(0)
filtered_similarity = similarity_matrix.masked_select(mask).view(embeddings.size(0), -1)
nn_sim = filtered_similarity.max(dim=1).values
quality = [item['quality'] for item in add_in_data_12]
complexity = [item['complexity'] for item in add_in_data_12]
print(f"Add in 12: quality: {sum(quality) / len(quality)} -- complexity: {sum(complexity) / len(complexity)} -- avg similarity: {filtered_similarity.mean()} -- avg nearest neighbour similarity: {nn_sim.mean()}")

Filtered out 12: quality: 4.403591043384498 -- complexity: 1.951808860616201 -- avg similarity: 0.752318799495697 -- avg nearest neighbour similarity: 0.8985397219657898
Add in 12: quality: 4.357931977857371 -- complexity: 2.825256210936916 -- avg similarity: 0.5447814464569092 -- avg nearest neighbour similarity: 0.82834792137146


In [21]:
embeddings = torch.cat([item['embedding'].unsqueeze(0) for item in filtered_out_data_23], dim=0).cuda()

similarity_matrix = torch.zeros((embeddings.size(0), embeddings.size(0)))
batch_size = 1  # 可以根据显存大小调整

# 逐批计算
for i in range(0, embeddings.size(0), batch_size):
    end_i = i + batch_size
    batch_i = embeddings[i:end_i]
    similarity = F.cosine_similarity(batch_i.unsqueeze(1), embeddings.unsqueeze(0), dim=-1)
    similarity_matrix[i:end_i, :] = similarity

mask = torch.ones(similarity_matrix.shape, dtype=bool)
mask.fill_diagonal_(0)
filtered_similarity = similarity_matrix.masked_select(mask).view(embeddings.size(0), -1)
nn_sim = filtered_similarity.max(dim=1).values
quality = [item['quality'] for item in filtered_out_data_23]
complexity = [item['complexity'] for item in filtered_out_data_23]
print(f"Filtered out 23: quality: {sum(quality) / len(quality)} -- complexity: {sum(complexity) / len(complexity)} -- avg similarity: {filtered_similarity.mean()} -- avg nearest neighbour similarity: {nn_sim.mean()}")

embeddings = torch.cat([item['embedding'].unsqueeze(0) for item in add_in_data_23], dim=0).cuda()

similarity_matrix = torch.zeros((embeddings.size(0), embeddings.size(0)))
batch_size = 1  # 可以根据显存大小调整

# 逐批计算
for i in range(0, embeddings.size(0), batch_size):
    end_i = i + batch_size
    batch_i = embeddings[i:end_i]
    similarity = F.cosine_similarity(batch_i.unsqueeze(1), embeddings.unsqueeze(0), dim=-1)
    similarity_matrix[i:end_i, :] = similarity

mask = torch.ones(similarity_matrix.shape, dtype=bool)
mask.fill_diagonal_(0)
filtered_similarity = similarity_matrix.masked_select(mask).view(embeddings.size(0), -1)
nn_sim = filtered_similarity.max(dim=1).values
quality = [item['quality'] for item in add_in_data_23]
complexity = [item['complexity'] for item in add_in_data_23]
print(f"Add in 23: quality: {sum(quality) / len(quality)} -- complexity: {sum(complexity) / len(complexity)} -- avg similarity: {filtered_similarity.mean()} -- avg nearest neighbour similarity: {nn_sim.mean()}")

Filtered out 23: quality: 4.357931977857371 -- complexity: 2.825256210936916 -- avg similarity: 0.5447814464569092 -- avg nearest neighbour similarity: 0.82834792137146
Add in 23: quality: 4.606013654295061 -- complexity: 4.015326088705995 -- avg similarity: 0.5823290944099426 -- avg nearest neighbour similarity: 0.83968186378479


In [22]:
embeddings = torch.cat([item['embedding'].unsqueeze(0) for item in filtered_out_data_34], dim=0).cuda()

similarity_matrix = torch.zeros((embeddings.size(0), embeddings.size(0)))
batch_size = 1  # 可以根据显存大小调整

# 逐批计算
for i in range(0, embeddings.size(0), batch_size):
    end_i = i + batch_size
    batch_i = embeddings[i:end_i]
    similarity = F.cosine_similarity(batch_i.unsqueeze(1), embeddings.unsqueeze(0), dim=-1)
    similarity_matrix[i:end_i, :] = similarity

mask = torch.ones(similarity_matrix.shape, dtype=bool)
mask.fill_diagonal_(0)
filtered_similarity = similarity_matrix.masked_select(mask).view(embeddings.size(0), -1)
nn_sim = filtered_similarity.max(dim=1).values
quality = [item['quality'] for item in filtered_out_data_34]
complexity = [item['complexity'] for item in filtered_out_data_34]
print(f"Filtered out 34: quality: {sum(quality) / len(quality)} -- complexity: {sum(complexity) / len(complexity)} -- avg similarity: {filtered_similarity.mean()} -- avg nearest neighbour similarity: {nn_sim.mean()}")

embeddings = torch.cat([item['embedding'].unsqueeze(0) for item in add_in_data_34], dim=0).cuda()

similarity_matrix = torch.zeros((embeddings.size(0), embeddings.size(0)))
batch_size = 1  # 可以根据显存大小调整

# 逐批计算
for i in range(0, embeddings.size(0), batch_size):
    end_i = i + batch_size
    batch_i = embeddings[i:end_i]
    similarity = F.cosine_similarity(batch_i.unsqueeze(1), embeddings.unsqueeze(0), dim=-1)
    similarity_matrix[i:end_i, :] = similarity

mask = torch.ones(similarity_matrix.shape, dtype=bool)
mask.fill_diagonal_(0)
filtered_similarity = similarity_matrix.masked_select(mask).view(embeddings.size(0), -1)
nn_sim = filtered_similarity.max(dim=1).values
quality = [item['quality'] for item in add_in_data_34]
complexity = [item['complexity'] for item in add_in_data_34]
print(f"Add in 34: quality: {sum(quality) / len(quality)} -- complexity: {sum(complexity) / len(complexity)} -- avg similarity: {filtered_similarity.mean()} -- avg nearest neighbour similarity: {nn_sim.mean()}")

Filtered out 34: quality: 4.606013654295061 -- complexity: 4.015326088705995 -- avg similarity: 0.5823290944099426 -- avg nearest neighbour similarity: 0.83968186378479
Add in 34: quality: 4.70867566511273 -- complexity: 4.295817918055113 -- avg similarity: 0.557906985282898 -- avg nearest neighbour similarity: 0.8309535384178162
